# Explore Book Rating Data

In [1]:
liked_books = ["53732", "117902", "472331", "6066095", "3850639", "526270", "11250317"]

First Steps:
1. Find all the users who like these same books (using the goodreads interaction file because it has user id's)
2. Then find all the books they like because we assume the have similar taste as ours. 

In [2]:
# We have to make sure the id's match up across files (like mentioned in search)

!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [3]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.split(",")
        csv_book_mapping[csv_id] = book_id

In [4]:
len(csv_book_mapping)

2360651

In [5]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


In [7]:
!ls -lh | grep goodreads_interactions

-rw-r--r--  1 mellanyandrea  staff   4.0G Feb 24 21:01 goodreads_interactions.csv


# Finding Users who Like the same books as Us

In [ ]:
# Pandas would use more memory so we won't use it. We are using streaming method. 

overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, rating, _ = line_split(",")
        
        if user_id in overlap_users:
            continue
            
        try:
            rating = int(rating)
        except ValueError:
            continue
            
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >=4:
            overlap_users.add(user_id)